# 1. Introducción

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

train = pd.read_csv("data/train.csv")
test = pd.read_csv("data/test.csv")

In [2]:
pd.set_option('display.max_columns', 1000) #ver todas las columnas

In [3]:
pd.set_option('display.max_rows', 100) #como mucho ver 100 filas

# 2. Preprocesamiento de los Datos

### Se quita id, titulo, descripción, direccion e Idzona por no entenderlos como tan relevantes

In [4]:
dfinicial = train
dftest = test

dfinicial = dfinicial.drop( columns = ['id', 'titulo', 'descripcion', 'direccion', 'idzona'] )
dftest = dftest.drop( columns = ['id', 'titulo', 'descripcion', 'direccion', 'idzona'] )

### Se quita latitud y longitud por la cantidad de NaN, manejando la granularidad sólo a nivel Ciudad y Provincia

In [5]:
dfinicial = dfinicial.drop( columns = ['lat', 'lng'] )
dftest = dftest.drop( columns = ['lat', 'lng'] )

### Se rellenan con la moda Baños, Garages y Habitaciones

In [6]:
def completarAlgunosNan2(df):
    for column in ["banos", "garages", "habitaciones"]:
        df[column] = df.groupby(['provincia', 'tipodepropiedad'])[column].apply(lambda x: x.fillna(x.mode()))
        df[column] = df[column].fillna(df[column].mode()[0]) #Por si son todos NaN, y no hay moda
        
    return df
    

dfinicial = completarAlgunosNan2(dfinicial)
dftest = completarAlgunosNan2(dftest)

### Se rellena Antigüedad, Metros Cubiertos y Metros totales con la mediana para los casos de NaN, por tener estos atributos distribución log normal (aproximadamente)

In [7]:
def completarAlgunosNan(df):
    for column in ['antiguedad', 'metroscubiertos', 'metrostotales']:
        df[column] = df.groupby(['provincia', 'tipodepropiedad'])[column].apply(lambda x: x.fillna(x.median()))
        df[column] = df[column].fillna(df[column].median())

    return df

dfinicial = completarAlgunosNan(dfinicial)
dftest = completarAlgunosNan(dftest)
        

/home/julian/.local/lib/python3.7/site-packages/numpy/lib/nanfunctions.py:1112: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)


### Aplicar Onehot encoding para desprenderse de provincias, ciudades tipo de propiedad como tal

In [8]:
# Se hallan los casos posibles
provincias_posibles = train["provincia"].dropna().unique()
props_posibles = train["tipodepropiedad"].dropna().unique()
#ciudades_posibles = train["ciudad"].dropna().unique()

# Se pasa tipodepropiedad y provincia a valores categóricos
dfinicial["tipodepropiedad"] = pd.Categorical(train["tipodepropiedad"], categories = props_posibles)
dfinicial["provincia"] = pd.Categorical(train["provincia"], categories = provincias_posibles)
#dfinicial["ciudad"] = pd.Categorical(train["ciudad"], categories = ciudades_posibles)

dftest["tipodepropiedad"] = pd.Categorical(test["tipodepropiedad"], categories=props_posibles)
dftest["provincia"] = pd.Categorical(test["provincia"], categories=provincias_posibles)
#dftest["ciudad"] = pd.Categorical(test["ciudad"], categories = ciudades_posibles)

# Rellenar nans de provincia, tipo de propiedad y ciudad, y usar onehot encoding
#datos_categoricos = dfinicial[["tipodepropiedad", "provincia", "ciudad"]].copy()
datos_categoricos = dfinicial[["tipodepropiedad", "provincia"]].copy()
datos_categoricos['tipodepropiedad'] = datos_categoricos.groupby(['provincia'])['tipodepropiedad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos["provincia"] = datos_categoricos["provincia"].fillna(datos_categoricos["provincia"].mode()[0])
#datos_categoricos["ciudad"] = datos_categoricos.groupby(['provincia'])['ciudad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos = pd.get_dummies(datos_categoricos, sparse = True)
dfinicial = pd.concat([datos_categoricos, dfinicial], axis=1)

#datos_categoricos = dftest[["tipodepropiedad", "provincia", "ciudad"]].copy()
datos_categoricos = dftest[["tipodepropiedad", "provincia"]].copy()
datos_categoricos['tipodepropiedad'] = datos_categoricos.groupby(['provincia'])['tipodepropiedad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos["provincia"] = datos_categoricos["provincia"].fillna(datos_categoricos["provincia"].mode()[0])
#datos_categoricos["ciudad"] = datos_categoricos.groupby(['provincia'])['ciudad'].apply(lambda x: x.fillna(x.mode()))
datos_categoricos = pd.get_dummies(datos_categoricos, sparse = True)
dftest = pd.concat([datos_categoricos, dftest], axis=1)

# Tirar tipodepropiedad y provincia
dfinicial = dfinicial.drop( columns = ['tipodepropiedad', 'provincia', 'ciudad'] )
dftest = dftest.drop( columns = ['tipodepropiedad', 'provincia', 'ciudad'] )

### Se pasa fecha a datetime y se agrega mes y año

In [9]:
dfinicial['fecha'] = dfinicial['fecha'].astype('datetime64')
dftest['fecha'] = dftest['fecha'].astype('datetime64')

dfinicial['dia'] = dfinicial['fecha'].dt.day
dfinicial['mes'] = dfinicial['fecha'].dt.month
dfinicial['anio'] = dfinicial['fecha'].dt.year
dfinicial = dfinicial.drop( columns = ['fecha'] )

dftest['dia'] = dftest['fecha'].dt.day
dftest['mes'] = dftest['fecha'].dt.month
dftest['anio'] = dftest['fecha'].dt.year
dftest = dftest.drop( columns = ['fecha'] )

### Adecuación de tipos de datos ahora que no hay NaN

In [10]:
 def parseoNan(df):
    df['gimnasio'] = df['gimnasio'].astype(np.uint8)
    df['usosmultiples'] = df['usosmultiples'].astype(np.uint8)
    df['piscina'] = df['piscina'].astype(np.uint8)
    df['escuelascercanas'] = df['escuelascercanas'].astype(np.uint8)
    df['centroscomercialescercanos'] = df['centroscomercialescercanos'].astype(np.uint8)
    
    df['garages'] = df['garages'].astype(np.uint8)
    df['antiguedad'] = df['antiguedad'].astype(np.uint8)
    df['banos'] = df['banos'].astype(np.uint8)
    df['habitaciones'] = df['habitaciones'].astype(np.uint8)
    
    df['metroscubiertos'] = df['metroscubiertos'].astype(np.uint16)
    df['metrostotales'] = df['metrostotales'].astype(np.uint16)
   
    df['dia'] = df['dia'].astype(np.uint8)
    df['mes'] = df['mes'].astype(np.uint8)
    df['anio'] = df['anio'].astype(np.uint16)
    
parseoNan(dfinicial)
parseoNan(dftest)

dfinicial['precio'] = dfinicial['precio'].astype(np.uint32)

In [11]:
'''### Se quitan columnas correspondientes a ciudades que sólo están asociadas a una o dos propiedades#

cant_ciudad = pd.DataFrame( train.groupby("ciudad").size(), columns = ["cantidad"] ).reset_index()
ciudad_pocas = cant_ciudad[ cant_ciudad["cantidad"] < 3 ]

ciudades = [ "ciudad_" + p for p in ciudad_pocas['ciudad'].to_list() ]

dfinicial = dfinicial.drop( columns = ciudades )
dftest = dftest.drop( columns = ciudades )'''

'### Se quitan columnas correspondientes a ciudades que sólo están asociadas a una o dos propiedades#\n\ncant_ciudad = pd.DataFrame( train.groupby("ciudad").size(), columns = ["cantidad"] ).reset_index()\nciudad_pocas = cant_ciudad[ cant_ciudad["cantidad"] < 3 ]\n\nciudades = [ "ciudad_" + p for p in ciudad_pocas[\'ciudad\'].to_list() ]\n\ndfinicial = dfinicial.drop( columns = ciudades )\ndftest = dftest.drop( columns = ciudades )'

In [12]:
dftest.to_csv(f"data/test_limpio.csv", index=False)

In [13]:
dfinicial.to_csv(f"data/train_limpio.csv", index=False)

In [14]:
dfinicial.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240000 entries, 0 to 239999
Data columns (total 71 columns):
tipodepropiedad_Apartamento                      -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Casa en condominio               -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Casa                             -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Terreno                          -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Terreno comercial                -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Local Comercial                  -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Quinta Vacacional                -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Oficina comercial                -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Edificio                         -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Casa uso de suelo                -240000 non-null Sparse[uint8, 0]
tipodepropiedad_Local en centro comercial        -240000 no